Required libraries:
    # Add python libraries. 
#Drivers for clickhouse database

conda install -c conda-forge clickhouse-driver

conda install -c conda-forge clickhouse-sqlalchemy

#pandas to clickhouse connector

pip install pandahouse

#Tinkoff's API implementation in python
pip install -i https://test.pypi.org/simple/ --extra-index-url=https://pypi.org/simple/ tinkoff-invest-openapi-client


In [1]:

from datetime import datetime, timedelta
from pytz import timezone
import pandas as pd
import time
import  ../src/data/CassandraHelper
import ../src/data/tinkoffAPIHelper

{'payload': {}, 'status': 'Ok', 'tracking_id': 'b1a5696aad3b899a'}

In [3]:
tick=get_instruments(instrument='stock')

In [4]:
tick

,figi,ticker,isin,min_price_increment,lot,currency,name,type
0,BBG000HLJ7M4,IDCC,US45867G1013,0.01,1,USD,InterDigItal Inc,Stock
1,BBG002293PJ4,RH,US74967X1037,0.01,1,USD,RH,Stock
2,BBG000CTQBF3,SBUX,US8552441094,0.01,1,USD,Starbucks Corporation,Stock
3,BBG000BPL8G3,MTSC,US5537771033,0.01,1,USD,MTS Systems Corp,Stock
4,BBG000BH0FR6,SGEN,US8125781026,0.01,1,USD,Seattle Genetics Inc,Stock
...,...,...,...,...,...,...,...,...
1280,BBG00K7T3037,BILI,US0900401060,0.01,1,USD,Bilibili Inc,Stock
1281,BBG000BG1DH3,COHR,US1924791031,0.01,1,USD,Coherent Inc,Stock
1282,BBG000BV4DR6,TER,US8807701029,0.01,1,USD,Teradyne Inc,Stock
1283,BBG000PSXT64,CPS,US21676P1030,0.01,1,USD,Cooper-Standard Holdings Inc,Stock


In [5]:
bond=get_instruments(instrument='bond')

In [6]:
bond

,figi,ticker,isin,min_price_increment,lot,currency,name,type
0,BBG00844BD08,RU000A0JU898,RU000A0JU898,0.1,1,RUB,МКБ выпуск 9,Bond
1,BBG00R05JT04,RU000A1013Y3,RU000A1013Y3,0.1,1,RUB,Черкизово выпуск 2,Bond
2,BBG00PNLY692,RU000A100DC4,RU000A100DC4,0.1,1,RUB,МСБ-Лизинг 002P выпуск 2,Bond
3,BBG00KHGQP89,RU000A0ZZ1F6,RU000A0ZZ1F6,0.1,1,RUB,КарМани выпуск 2,Bond
4,BBG00JYHHXN7,RU000A0ZYR18,RU000A0ZYR18,0.1,1,RUB,Башкирская содовая компания выпуск 1,Bond
...,...,...,...,...,...,...,...,...
378,BBG00QYKX940,RU000A1012K4,RU000A1012K4,0.1,1,RUB,СЭЗ им. Серго Орджоникидзе,Bond
379,BBG00BQZW6R1,RU000A0JW1K9,RU000A0JW1K9,0.1,1,RUB,Томская область выпуск 55,Bond
380,BBG00PVVTDW6,RU000A100K64,RU000A100K64,0.1,1,RUB,Флай Плэнинг БО-П2,Bond
381,BBG00PC4M4X3,RU000A100DG5,RU000A100DG5,0.1,1,RUB,Европлан выпуск 6,Bond


In [7]:
etf=get_instruments(instrument='etf')


In [8]:
etf

,figi,ticker,isin,min_price_increment,lot,currency,name,type
0,BBG005DXDPK9,FXGD,IE00B8XB7377,0.2000,1,RUB,Золото,Etf
1,BBG00M8C8Y03,FXKZ,IE00BG0C3K84,0.1000,1,RUB,Акции казахстанских компаний,Etf
2,BBG005VKB7D7,FXCN,IE00BD3QFB18,2.0000,1,RUB,Акции китайских компаний,Etf
3,BBG000000000,TUSD,RU000A1011S9,0.0001,1,USD,Вечный портфель USD,Etf
4,BBG000000002,TEUR,RU000A1011T7,0.0001,1,EUR,Вечный портфель EUR,Etf
5,BBG000000001,TRUR,RU000A1011U5,0.0050,1,RUB,Вечный портфель RUB,Etf
6,BBG00KMXH2C2,RUSB,LU1483649825,0.5000,1,RUB,Долларовые еврооблигации российских эмитентов,Etf
7,BBG005HLSZ23,FXUS,IE00BD3QHZ91,1.0000,1,RUB,Акции американских компаний,Etf
8,BBG0073DLHS1,FXRL,IE00BQ1Y6480,0.5000,1,RUB,Акции российских компаний,Etf
9,BBG005N1KZX4,FXRU,IE00BD5FH213,0.1000,1,RUB,Еврооблигации российских компаний (USD),Etf


In [37]:
from tqdm import tqdm
def get_detailed_data(data,_from=None,to=None):
    list_df=[]
    for figi in tqdm(data.figi.unique()):
        print("retrieving figi: "+figi+ " ticker: "+data[data.figi==figi].ticker.values)
        
        df=detailed_history(figi=figi,
                                                            _from=_from,
                                                            to=to,
                                                            interval="1min")
        time.sleep(3)
        list_df.append(df)
        print(f'Len list of df {len(list_df)}')
    
    df_merge = pd.concat(list_df)
    return df_merge

In [31]:
from datetime import datetime
days_span=90
startTime = datetime.now() - timedelta(days=days_span)
endTime = datetime.now()


In [ ]:
tick_data=get_detailed_data(tick,_from=startTime,to=endTime)

In [38]:
etf_data=get_detailed_data(etf,_from=startTime,to=endTime)

  0%|                                                                                           | 0/24 [00:00<?, ?it/s]

['retrieving figi: BBG005DXDPK9 ticker: FXGD']
Concat frame 0. Dataframe with data (row,col) : (521, 8).
Concat frame 1. Dataframe with data (row,col) : (521, 8).
Concat frame 2. Dataframe with data (row,col) : (521, 8).
Concat frame 3. Dataframe with data (row,col) : (1043, 8).
Concat frame 4. Dataframe with data (row,col) : (1566, 8).
Concat frame 5. Dataframe with data (row,col) : (2088, 8).
Concat frame 6. Dataframe with data (row,col) : (2611, 8).
Concat frame 7. Dataframe with data (row,col) : (3134, 8).
Concat frame 8. Dataframe with data (row,col) : (3134, 8).
Concat frame 9. Dataframe with data (row,col) : (3134, 8).
Concat frame 10. Dataframe with data (row,col) : (3656, 8).
Concat frame 11. Dataframe with data (row,col) : (4175, 8).
Concat frame 12. Dataframe with data (row,col) : (4700, 8).
Concat frame 13. Dataframe with data (row,col) : (5223, 8).
Concat frame 14. Dataframe with data (row,col) : (5748, 8).
Concat frame 15. Dataframe with data (row,col) : (5748, 8).
Concat

  4%|███▍                                                                               | 1/24 [00:16<06:30, 16.98s/it]

Len list of df 1
['retrieving figi: BBG00M8C8Y03 ticker: FXKZ']
Concat frame 0. Dataframe with data (row,col) : (240, 8).
Concat frame 1. Dataframe with data (row,col) : (240, 8).
Concat frame 2. Dataframe with data (row,col) : (240, 8).
Concat frame 3. Dataframe with data (row,col) : (420, 8).
Concat frame 4. Dataframe with data (row,col) : (594, 8).
Concat frame 5. Dataframe with data (row,col) : (784, 8).
Concat frame 6. Dataframe with data (row,col) : (985, 8).
Concat frame 7. Dataframe with data (row,col) : (1182, 8).
Concat frame 8. Dataframe with data (row,col) : (1182, 8).
Concat frame 9. Dataframe with data (row,col) : (1182, 8).
Concat frame 10. Dataframe with data (row,col) : (1390, 8).
Concat frame 11. Dataframe with data (row,col) : (1596, 8).
Concat frame 12. Dataframe with data (row,col) : (1834, 8).
Concat frame 13. Dataframe with data (row,col) : (2063, 8).
Concat frame 14. Dataframe with data (row,col) : (2310, 8).
Concat frame 15. Dataframe with data (row,col) : (231

  8%|██████▉                                                                            | 2/24 [00:28<05:37, 15.33s/it]

...

Len list of df 22
['retrieving figi: BBG00NRFC2X2 ticker: FXTB']
Concat frame 0. Dataframe with data (row,col) : (427, 8).
Concat frame 1. Dataframe with data (row,col) : (427, 8).
Concat frame 2. Dataframe with data (row,col) : (427, 8).
Concat frame 3. Dataframe with data (row,col) : (873, 8).
Concat frame 4. Dataframe with data (row,col) : (1253, 8).
Concat frame 5. Dataframe with data (row,col) : (1713, 8).
Concat frame 6. Dataframe with data (row,col) : (2140, 8).
Concat frame 7. Dataframe with data (row,col) : (2569, 8).
Concat frame 8. Dataframe with data (row,col) : (2569, 8).
Concat frame 9. Dataframe with data (row,col) : (2569, 8).
Concat frame 10. Dataframe with data (row,col) : (3038, 8).
Concat frame 11. Dataframe with data (row,col) : (3505, 8).
Concat frame 12. Dataframe with data (row,col) : (4005, 8).
Concat frame 13. Dataframe with data (row,col) : (4433, 8).
Concat frame 14. Dataframe with data (row,col) : (4918, 8).
Concat frame 15. Dataframe with data (row,col) : 

 96%|██████████████████████████████████████████████████████████████████████████████▌   | 23/24 [05:07<00:12, 12.96s/it]

Len list of df 23
['retrieving figi: BBG005HLK5V5 ticker: FXUK']
Concat frame 0. Dataframe with data (row,col) : (0, 0).
Concat frame 1. Dataframe with data (row,col) : (0, 0).
Concat frame 2. Dataframe with data (row,col) : (0, 0).
Concat frame 3. Dataframe with data (row,col) : (0, 0).
Concat frame 4. Dataframe with data (row,col) : (0, 0).
Concat frame 5. Dataframe with data (row,col) : (0, 0).
Concat frame 6. Dataframe with data (row,col) : (0, 0).
Concat frame 7. Dataframe with data (row,col) : (0, 0).
Concat frame 8. Dataframe with data (row,col) : (0, 0).
Concat frame 9. Dataframe with data (row,col) : (0, 0).
Concat frame 10. Dataframe with data (row,col) : (0, 0).
Concat frame 11. Dataframe with data (row,col) : (0, 0).
Concat frame 12. Dataframe with data (row,col) : (0, 0).
Concat frame 13. Dataframe with data (row,col) : (0, 0).
Concat frame 14. Dataframe with data (row,col) : (0, 0).
Concat frame 15. Dataframe with data (row,col) : (0, 0).
Concat frame 16. Dataframe with d

100%|██████████████████████████████████████████████████████████████████████████████████| 24/24 [05:17<00:00, 13.22s/it]

Len list of df 24


In [41]:
etf_data.shape

(422051, 8)

In [ ]:
bonds_data=get_detailed_data(bond,_from=startTime,to=endTime)

In [ ]:
def set_balance():
    balance_set = client.sandbox.sandbox_currencies_balance_post({"currency": "USD", "balance": 10000})
    print("balance")
    print(balance_set)
    print()


def print_24hr_operations():
    now = datetime.now(tz=timezone('Europe/Moscow'))
    yesterday = now - timedelta(days=1)
    ops = client.operations.operations_get(_from=yesterday.isoformat(), to=now.isoformat())
    print("operations")
    print(ops)
    print()


def print_orders():
    orders = client.orders.orders_get()
    print("active orders")
    print(orders)
    print()


def make_order():
    order_response = client.orders.orders_limit_order_post(figi='BBG009S39JX6',
                                                           limit_order_request={"lots": 1,
                                                                                "operation": "Buy",
                                                                                "price": 0.01})
    print("make order")
    print(order_response)
    print()
    return order_response


# won't work in sandbox - orders are being instantly executed
def cancel_order(order_id):
    cancellation_result = client.orders.orders_cancel_post(order_id=order_id)
    print("cancel order")
    print(cancellation_result)
    print()


set_balance()
print_24hr_operations()
print_orders()
order_response = make_order()
print_orders()
# cancel_order(order_response.payload.order_id)
# print_orders()